In [1]:
%env SPECTRAL_CONNECTIVITY_ENABLE_GPU=true

env: SPECTRAL_CONNECTIVITY_ENABLE_GPU=true


In [2]:
import sys
sys.path.append("../../lfp")

In [3]:
import cupy as xp
from cupyx.scipy.fft import ifft
from cupyx.scipy.sparse.linalg import svds
from spectral_connectivity import Multitaper, Connectivity
import LFP_recording as LFP_recording
import LFP_collection as LFP_collection
import preprocessor as preprocessor
import importlib
import pandas as pd
from itertools import combinations
import os
from bidict import bidict
import pickle

def hex_2_rgb(hex_color): # Orange color
    rgb_color = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (1, 3, 5))
    return rgb_color

def pickle_this(thing_to_pickle, file_name):
    """
    Pickles things
    Args (2):   
        thing_to_pickle: anything you want to pickle
        file_name: str, filename that ends with .pkl 
    Returns:
        none
    """
    with open(file_name,'wb') as file:
        pickle.dump(thing_to_pickle, file)

def unpickle_this(pickle_file):
    """
    Unpickles things
    Args (1):   
        file_name: str, pickle filename that already exists and ends with .pkl
    Returns:
        pickled item
    """
    with open(pickle_file, 'rb') as file:
        return(pickle.load(file))
    


TRYING GPU!
THIS IS GPU
c:\Users\Padilla-Coreano\.conda\envs\ephy_analysis\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
c:\Users\Padilla-Coreano\.conda\envs\ephy_analysis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
trodes_directory = r'C:\Users\Padilla-Coreano\Desktop\Trodes_2-5-3_Windows64'

df = pd.read_excel(r"C:\Users\Padilla-Coreano\Desktop\GITHUB_REPOS\diff_fam_social_memory_ephys\lfp\channel_mapping_sme.xlsx")
spike_cols = [col for col in df.columns if 'spike_interface_' in col.lower()]

# Extract brain regions from column names
# Assumes format 'spike_interface_REGION'
brain_regions = [col.split('spike_interface_')[1] for col in spike_cols]

# Create nested dictionary
subject_to_channel_dict = {}

for _, row in df.iterrows():
    subject = row['Subject'].astype(str)
    # Initialize inner dictionary for this subject
    subject_to_channel_dict[subject] = {}
    
    # Populate inner dictionary with brain region: spike value pairs
    for col, region in zip(spike_cols, brain_regions):
        subject_to_channel_dict[subject][region] = int(row[col])
behavior_dicts = {}
def make_recording_to_subj_dict(data_path):
    recording_to_subject = {}
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith('merged.rec'):
                subject = str(int((file.split("_")[0]))/10)
                recording_to_subject[file] = subject
                behavior_dicts[file] = {}
    return recording_to_subject


def process(data_path):
    recording_to_subject = make_recording_to_subj_dict(data_path)
    print(recording_to_subject)
    collection = LFP_collection.LFPCollection(behavior_dicts, subject_to_channel_dict, data_path, recording_to_subject, 4, trodes_directory)
    #collection.process()
    return collection
    
    
data_path = r"C:\Users\Padilla-Coreano\UFL Dropbox\Caroline De Paula Cunha Almeida\Padilla-Coreano Lab\2024\Cum_SocialMemEphys_pilot2\Object_Control (phase 8)\data"
pickle_path = r"C:\Users\Padilla-Coreano\UFL Dropbox\Caroline De Paula Cunha Almeida\Padilla-Coreano Lab\2024\Cum_SocialMemEphys_pilot2\processed_lfp_pickles\object_control_p8_processed_lfp.pkl"
collection = process(data_path)


{'22_object_merged.rec': '2.2', '44_object_merged.rec': '4.4', '23_object_merged.rec': '2.3', '41_object_merged.rec': '4.1', '31_object_merged.rec': '3.1', '32_object_merged.rec': '3.2'}
Processing 22_object_merged.rec


c:\Users\Padilla-Coreano\Desktop\GITHUB_REPOS\diff_fam_social_memory_ephys\lfp\lfp_analysis\../../lfp\trodes\read_exported.py:167: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)


Extracted first timestamp
Processing 44_object_merged.rec
Extracted first timestamp
Processing 23_object_merged.rec
Extracted first timestamp
Processing 41_object_merged.rec
Extracted first timestamp
Processing 31_object_merged.rec
Extracted first timestamp
Processing 32_object_merged.rec
Extracted first timestamp


In [5]:
collection.process()


  0%|          | 0/6 [00:00<?, ?it/s]

processing 22_object_merged.rec
RMS Traces calculated
Power Calculated
Coherence calcualatd


Maximum iterations reached. 2852 of 3620 converged
Maximum iterations reached. 2984 of 3620 converged
Maximum iterations reached. 2779 of 3620 converged
Maximum iterations reached. 2768 of 3620 converged
Maximum iterations reached. 2816 of 3620 converged
Maximum iterations reached. 2711 of 3620 converged
Maximum iterations reached. 2576 of 3620 converged
Maximum iterations reached. 2785 of 3620 converged
Maximum iterations reached. 2794 of 3620 converged
Maximum iterations reached. 2546 of 3620 converged
 17%|█▋        | 1/6 [05:03<25:16, 303.33s/it]

Granger's causality calculated
processing 44_object_merged.rec
RMS Traces calculated
Power Calculated
Coherence calcualatd


Maximum iterations reached. 3252 of 3618 converged
Maximum iterations reached. 3312 of 3618 converged
Maximum iterations reached. 3182 of 3618 converged
Maximum iterations reached. 3036 of 3618 converged
Maximum iterations reached. 3331 of 3618 converged
Maximum iterations reached. 3215 of 3618 converged
Maximum iterations reached. 3040 of 3618 converged
Maximum iterations reached. 3331 of 3618 converged
Maximum iterations reached. 3227 of 3618 converged
Maximum iterations reached. 3048 of 3618 converged
 33%|███▎      | 2/6 [10:05<20:10, 302.62s/it]

...

Granger's causality calculated
processing 31_object_merged.rec
RMS Traces calculated
Power Calculated
Coherence calcualatd


Maximum iterations reached. 3354 of 3659 converged
Maximum iterations reached. 2933 of 3659 converged
Maximum iterations reached. 3211 of 3659 converged
Maximum iterations reached. 3033 of 3659 converged
Maximum iterations reached. 2983 of 3659 converged
Maximum iterations reached. 3282 of 3659 converged
Maximum iterations reached. 3026 of 3659 converged
Maximum iterations reached. 2859 of 3659 converged
Maximum iterations reached. 2577 of 3659 converged
Maximum iterations reached. 2889 of 3659 converged
 83%|████████▎ | 5/6 [26:07<05:18, 318.24s/it]

Granger's causality calculated
processing 32_object_merged.rec
RMS Traces calculated
Power Calculated
Coherence calcualatd


Maximum iterations reached. 2777 of 3660 converged
Maximum iterations reached. 2749 of 3660 converged
Maximum iterations reached. 2713 of 3660 converged
Maximum iterations reached. 2655 of 3660 converged
Maximum iterations reached. 2671 of 3660 converged
Maximum iterations reached. 2629 of 3660 converged
Maximum iterations reached. 2444 of 3660 converged
Maximum iterations reached. 2589 of 3660 converged
Maximum iterations reached. 2350 of 3660 converged
Maximum iterations reached. 2396 of 3660 converged
100%|██████████| 6/6 [31:41<00:00, 316.88s/it]

Granger's causality calculated


In [6]:
pickle_this(collection, pickle_path)